In [1]:
import requests
import pandas as pd
import re
from psaw import PushshiftAPI
api = PushshiftAPI()
from datetime import datetime
import json
import pysolr
from tqdm import tqdm

In [2]:
subreddits = ['science','worldnews','news','food','askscience','space','mildlyinteresting','history','UpliftingNews'
             'philosophy','technology','politics']
def getSubmissionsBySubReddit(subreddit,limit):
    gen = api.search_submissions(subreddit=[subreddit], 
                             filter=['id','url','author', 'title', 'score',
                                       'subreddit','selftext','num_comments'], 
                             limit = limit, 
                             num_comments=">1")
    submission_list = list(gen)
    submission_list = [item.d_ for item in submission_list]
    submission_df = pd.DataFrame.from_dict(submission_list)
    submission_df = submission_df.fillna("")
    return convertSubmissionsTofinalSchema(submission_df,topic=None)

def getSubmissionsBySubRedditsAndTopic(subreddits,topic,limit,keyword):
    gen = api.search_submissions(subreddit=subreddits,
                             filter=['id','url','author', 'title', 'score',
                                       'subreddit','selftext','num_comments'], 
                             limit = limit, 
                             num_comments=">10", q=keyword) 

    submission_list = list(gen)
    submission_list = [item.d_ for item in submission_list]
    submission_df = pd.DataFrame.from_dict(submission_list)
    submission_df = submission_df.fillna("")
    return convertSubmissionsTofinalSchema(submission_df,topic=topic)

def convertSubmissionsTofinalSchema(submission_df,topic):
    submission_df = submission_df[submission_df.selftext!='[removed]']    
    submission_df = submission_df[submission_df.selftext!='[deleted]']
    
    submission_df['is_submission'] = True
    submission_df['selftext'] = submission_df['title']
    submission_df = submission_df[submission_df.selftext!='']
    
    submission_df['body'] = submission_df['selftext']
    submission_df['parent_id'] = None
    submission_df['parent_body'] = None
    submission_df['topic'] = topic
    submission_df=submission_df.rename({'url':'full_link'},axis=1)
    submission_df=submission_df.rename({'score':'upvotes'},axis=1)
    res= submission_df[['id','subreddit','full_link','title','body','selftext','author','is_submission','parent_id'
                          ,'parent_body','topic','created_utc','upvotes']]
    return res

def convertCommentsToFinalSchema(comments_df,topic=None):
    comments_df = comments_df[comments_df.body!='[removed]']    
    comments_df = comments_df[comments_df.body!='[deleted]']
    comments_df = comments_df[comments_df.body!='']
    comments_df['is_submission'] = False
    comments_df['topic'] = topic
    comments_df['parent_body'] = None
    comments_df['selftext'] = None
    comments_df['title'] = None
    comments_df['parent_id'] = comments_df['parent_id'].apply(lambda x:x.split('_')[1] )
    comments_df=comments_df.rename({'permalink':'full_link'},axis=1)
    res= comments_df[['id','subreddit','full_link','title','body','selftext','author','is_submission','parent_id'
                          ,'parent_body','topic','created_utc','score']]
    return res

def fetchCommentsForSubmission(submission):
    comments_lst = list(api.search_comments(link_id=submission["id"].iloc[0], subreddit=[submission["subreddit"].iloc[0]],
                                        filter=['id','parent_id','permalink','author', 'title', 
                                                'subreddit','body','num_comments','score']))
    
    comments_lst = [item.d_ for item in comments_lst]
    comments_df = pd.DataFrame.from_dict(comments_lst)
    comments_df = comments_df.fillna("")
    
    if len(comments_lst) == 0:
        return pd.DataFrame()
    
    if 'permalink' not in comments_df.columns:
#         print('no permalink')
        return pd.DataFrame()
    
    return convertCommentsToFinalSchema(comments_df,topic=submission["topic"].iloc[0])

def populateComments(submission):
    comments = fetchCommentsForSubmission(submission)
    if comments.size == 0:
        return pd.concat([submission])
    res = pd.concat([submission,comments])
    merged = pd.merge(res,res,left_on='parent_id',right_on='id',how='left')
    merged = merged.rename({'id_x':'id','subreddit_x':'subreddit','full_link_x':'full_link',
                        'title_x':'title','body_x':'body','selftext_x':'selftext','author_x':'author',
                        'is_submission_x':'is_submission','parent_id_x':'parent_id',
                        'topic_x':'topic','body_y':'parent_body','created_utc_x':'created_utc','score_x':'upvotes'},axis=1)
    final= merged[['id','subreddit','full_link','title','body','selftext','author','is_submission','parent_id'
                          ,'parent_body','topic','created_utc','upvotes']]
    # final = final.set_index('id')
    
    return final

In [8]:
submission_df = pd.concat([ 
getSubmissionsBySubRedditsAndTopic(subreddits,topic='Politics',limit=120,keyword='Democrats'),
getSubmissionsBySubRedditsAndTopic(subreddits,topic='Politics',limit=120,keyword='Republican'),
getSubmissionsBySubRedditsAndTopic(subreddits,topic='Environment',limit=120,keyword='Environment'),
getSubmissionsBySubRedditsAndTopic(subreddits,topic='Environment',limit=120,keyword='Climate change'),
getSubmissionsBySubRedditsAndTopic(subreddits,topic='Environment',limit=120,keyword='Solar energy'),
getSubmissionsBySubRedditsAndTopic(subreddits,topic='Technology',limit=120,keyword='Programming'),
getSubmissionsBySubRedditsAndTopic(subreddits,topic='Technology',limit=120,keyword='Computer'),
getSubmissionsBySubRedditsAndTopic(subreddits,topic='Technology',limit=120,keyword='Blockchain'),
getSubmissionsBySubRedditsAndTopic(subreddits,topic='Healthcare',limit=120,keyword='Healthcare'),
getSubmissionsBySubRedditsAndTopic(subreddits,topic='Healthcare',limit=120,keyword='Covid'),
getSubmissionsBySubRedditsAndTopic(subreddits,topic='Education',limit=120,keyword='Education'),
getSubmissionsBySubRedditsAndTopic(subreddits,topic='Education',limit=120,keyword='Learning'),
getSubmissionsBySubRedditsAndTopic(subreddits,topic='Education',limit=120,keyword='School'),
getSubmissionsBySubReddit('ExplainLikeImFive',150),
getSubmissionsBySubReddit('FoodForThought',150),
getSubmissionsBySubReddit('ChangeMyView',150),
getSubmissionsBySubReddit('TodayILearned',150)
])
print(submission_df.shape)

In [9]:
%%time
submissionAndComment = pd.DataFrame()
for i in tqdm(range(0,submission_df.shape[0])):
    submission = submission_df.iloc[[i]]
    final = populateComments(submission)
    submissionAndComment = pd.concat([submissionAndComment,final])

In [13]:
submissionAndComment['upvotes'] = submissionAndComment['upvotes'].fillna(1)

In [ ]:
submissionAndComment.to_csv('D:\\scrape2.csv')

In [ ]:
print(submissionAndComment.topic.unique())
print(submissionAndComment.upvotes.dtype)
print(submissionAndComment.upvotes.isna().unique())

In [18]:
submissionAndComment[(submissionAndComment.is_submission==False) & (submissionAndComment.parent_body.isna())]

,id,subreddit,full_link,title,body,selftext,author,is_submission,parent_id,parent_body,topic,created_utc,upvotes
29,i1f10fo,politics,/r/politics/comments/thjry8/ohio_republicans_w...,None,sooo...yes?,None,tokikain,False,i1ecz5i,NaN,Politics,1647792448,0.0
33,i1eww16,politics,/r/politics/comments/thjry8/ohio_republicans_w...,None,so there's no insecurity? none at all?.....so ...,None,tokikain,False,i1ecwaj,NaN,Politics,1647790724,1.0
99,i1c87ge,politics,/r/politics/comments/thjry8/ohio_republicans_w...,None,you dont represent the USA. its you that WE do...,None,[deleted],False,i19v3py,NaN,Politics,1647731411,3.0
147,i1bca5p,politics,/r/politics/comments/thjry8/ohio_republicans_w...,None,Youre sort of a piece shit. Im sure youre use...,None,rif011412,False,i19v3py,NaN,Politics,1647717340,2.0
221,i1acuqd,politics,/r/politics/comments/thjry8/ohio_republicans_w...,None,...said Vlad.,None,sngle1now2020,False,i19v3py,NaN,Politics,1647702373,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39119,gicqjpw,politics,/r/politics/comments/kryi79/megathread_us_capi...,None,Source,None,AnakinKardashian,False,gicqf2h,NaN,Politics,1609971268,1.0
39121,gicqjjx,politics,/r/politics/comments/kryi79/megathread_us_capi...,None,That's him and his wife. Check on desktop.,None,whatisevenrealnow,False,gicqeh8,NaN,Politics,1609971266,1.0
39146,gicqhvq,politics,/r/politics/comments/kryi79/megathread_us_capi...,None,No he didn't it's him and his wife lmao.,None,Seasniffer,False,gicqeh8,NaN,Politics,1609971246,1.0
39205,gicqcb9,politics,/r/politics/comments/kryi79/megathread_us_capi...,None,This is technically sedition,None,minimallyautistic,False,gicq4xq,NaN,Politics,1609971179,1.0


In [21]:
sum(submissionAndComment.is_submission == False)

1087961